### 1. Gather Data
Leverage the Google Maps Geocoding API to identify target companies and extract essential details. Store the collected information in a .csv file.

In this step, I gather data on the companies I'm interested in (in this example: electricians). Since I need information for the entire region of Flanders in Belgium, I based my search on cities, geographically distributed in this region with a search radios set to 10,000m. This way i cover the entire region.

The gathered data includes: Name, Website, Phone number and City. It is not possible in this step to scrape e-mail addresses as this is not supported by the Google API. See next steps for adding email addresses.

In [8]:
# Import the necessary libraries
import googlemaps
import pandas as pd
import csv
import time
import os

In [9]:
# Initialize Google Maps API client
API_KEY = input("your api key: ")
gmaps = googlemaps.Client(key=API_KEY)

# You will need a Google Cloud account to activate the Geocode API and the Maps Platform API:
# Generate your Google maps API Key and fill in where you encounter `API_KEY = your api key`
# See https://developers.google.com/maps/documentation/maps-static for more information.*

your api key: 

##### I made a list here containing cities and communities in the region of Flanders (Belgium), geographically distributed. The search radius is set to 10,000 meters to cover the entire region.

`"Kortrijk","Ieper","Alveringem","Koksijde","Diksmuide", "Poperinge","Roeselare", "Torhout","Lichtervelde", "Koekelare", "Gistel","Middelkerke","Oostende","De Haan", "Knokke", "Brugge", "Oostkamp","Zedelgem", "Wingene", "Tielt","Meulebeke","Waregem","Deerlijk","Anzegem", "Kruisem","aalter","maldegem","eeklo","evergem","gent","de pinte","nazareth","oosterzele","zottegem","oudenaarde","ninove","dendermonde","lokeren","sint-niklaas","stekene","beveren","zele","zelzate","lochristi","geraardsbergen","brakel",         "antwerpen","boom","duffel","essen","stabroek","wuustwezel","malle","turnhout","ravels","kasterlee","dessel","geel","herentals","grobbendonk","herselt","heist-op-den-berg","balen",
"tollembeek","gooik","pepingen","roosdaal","ternat","asse","sint-pieters-leeuw","sint-genesius-rode","halle","hoeilaart","tervuren","zaventem","merchtem","vilvoorde","londerzeel","haacht","leuven","holsbeek","glabbeek","diest","aarschot","sint-truiden","heusden-zolder","lommel","lummen","genk","oudsbergen","bilzen","hasselt","peer"`

In [27]:
# List of cities in the province of West-Flanders, Belgium

CITIES = input("Location(s) to search (comma-separated): ").split(',')

Location(s) to search (comma-separated): Kortrijk, Brugge


In [39]:
def search_places(query, location):
    places_result = gmaps.places(query=query, location=location, radius=10000, language='nl')
    return places_result['results']

# Prepare data for CSV output
all_data = []

for city in CITIES:
    city = city.strip()  # Remove any leading/trailing whitespace
    # Geocode the city
    geocode_result = gmaps.geocode(f"{city}, Belgium")
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
    else:
        print(f"Geocoding failed for {city}, Belgium")
        continue  # Skip to the next city if geocoding fails

    # Search for "electrician" in the city
    search_query = "elektricien"
    places = search_places(search_query, location)

    for place in places:
        name = place.get('name', 'n/a')
        website = 'n/a'
        phone_number = 'n/a'

        # Extract website and phone number from place details (if available)
        try:
            place_details = gmaps.place(place_id=place['place_id'], language='nl')
            if 'website' in place_details['result']:
                website = place_details['result']['website']
            if 'formatted_phone_number' in place_details['result']:
                phone_number = place_details['result']['formatted_phone_number']
        except Exception as e:
            print(f"Error getting place details for {name}: {e}")

        all_data.append([name, website, phone_number, city])  # Add city to data

# Write data to CSV file

# Specify the directory where you want to save the CSV file
directory = "gathered-data" #directory path

# Ensure the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

csv_file_path = os.path.join(directory,"elektriciens-vlaanderen.csv")
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Name', 'Website', 'Phone Number', 'City'])  # Add city header
    writer.writerows(all_data)

print(f"Data saved to {csv_file_path}")


Data saved to 1-gathered-data\elektriciens-vlaanderen.csv
